## Extract info from 120a budget report

E. Quinn 10/25/2019

This notebook uses pdfminer to extract the content of the MUNIS 120a budget report

The documentation for pdfminer is at:

https://buildmedia.readthedocs.org/media/pdf/pdfminer-docs/latest/pdfminer-docs.pdf

## Import standard python datascience packages

In [1]:
import math
import re
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Import pdfminer packages

In [2]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

### Show the directory we are running in

In [3]:
!pwd

/home/gquinn/EG/school_committee/egsc/notebooks


## Perform layout analysis - see section 2.3 of the pdfminer documentation


### Read the pdf and create a document object

In [5]:
document = open('../../finance_subcommittee/FY2019_budget/120a ny colems dept.pdf', 'rb')

### Create a resource manager object

In [6]:
rsrcmgr = PDFResourceManager()

### Set the parameters for analysis

In [7]:
laparams = LAParams()

### Create a PDF page aggregator object

In [8]:
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

### Store the information in a dictionary

Strategy for this document:  

Save information from each element in the LTTextBox objects in a dictionary including:

- x0 horizontal coordinate of the upper left corner of the text box
- x1 horizontal coordinate of the lower right corner of the text box
- page number 
- sequence number of text box within this page
- text contained in the text box, converted to ascii

Parsing the text is complicated by the fact that that a text box may span multiple columns and/or rows, and the text box groupings vary quite a bit depending on the page contents and layout.

However, with a bit of luck the structure of the document will allow the contents to be deciphered with the following heuristics:

- Text boxes containing left justified columns will tend to have nearly the same x0 coordinates
- Text boxes containing right justified columns will tend to have nearly the same x1 coordinates
- The codes for fund, account code, and object code are numeric and have fixed lengths
- Extraneous information is often preceded or followed by a series of underscore and newline characters
- Last name can be distinguished because is the only field that is all characters followed by a comma
- Last name may be preceded by between one and three numerical fields:  fund, account, object.  If it is, the x0 value is shifted to the left.
    - Three numerical fields precede the name:  assume they are fund, account, object
    - Two numerical fields precede the name: assume they are account, object
    - One numerical field precedes the name: assume it is object

In [9]:
pdf={}                                     #dictionary to hold the results

pageno = 0                                 #initialize page coounter to zero

for page in PDFPage.get_pages(document):   #loop through the pdf page by page
    pageno = pageno + 1                    #increment the page number
    pdf[pageno] = {}                       #dictionary for this page
    interpreter.process_page(page)         # receive the LTPage object for the page.
    layout = device.get_result()           # create layout object
    tbox_no=0                              # index for element number
    for element in layout:
        if (type(element).__name__=='LTTextBoxHorizontal'):             #loop through text boxes
            tbox_no += 1                                                #increment text box number
            pdf[pageno][tbox_no] = {}                                   #dictionary for text boxes within page
            x0 = round(element.x0,2)                                    #x0 coordinate of textbox corner
            x1 = round(element.x1,2)                                    #x1 coordinate of textbox corner
            txt = element.get_text().encode('ascii', 'ignore')          #text converted to ascii
            pdf[pageno][tbox_no]['x0'] = x0                             #create x0 coordinate entry
            pdf[pageno][tbox_no]['x1'] = x1                             #create x1 coordinate entry
            pdf[pageno][tbox_no]['text'] = txt                          #create text entry

### Show the results for the first two pages

In [10]:
pdf[1]     #page 1

{1: {'x0': 26.4, 'x1': 67.43, 'text': b'3/9/2018\n11:24:52AM\n'},
 2: {'x0': 26.4,
  'x1': 166.79,
  'text': b'04103 - Archie R. Cole Middle School\n'},
 3: {'x0': 310.95, 'x1': 468.0, 'text': b'EAST GREENWICH PUBLIC SCHOOL\n'},
 4: {'x0': 284.95,
  'x1': 493.88,
  'text': b'SCHOOL PROGRAM FY2019 PROPOSED BUDGET \n'},
 5: {'x0': 696.1,
  'x1': 770.52,
  'text': b'Page 1 of 13\nReport 120A\nIncl Non-Discretionary\n'},
 6: {'x0': 33.15, 'x1': 49.6, 'text': b'Fund\n'},
 7: {'x0': 65.15, 'x1': 77.59, 'text': b'Loc\n'},
 8: {'x0': 93.6, 'x1': 109.15, 'text': b'Dept\n'},
 9: {'x0': 123.85,
  'x1': 302.83,
  'text': b'Prog Gr/Sub Class Object Account Description\n'},
 10: {'x0': 381.4, 'x1': 397.4, 'text': b'2016\n'},
 11: {'x0': 373.1, 'x1': 405.54, 'text': b'ACTUAL\n'},
 12: {'x0': 444.4, 'x1': 460.4, 'text': b'2017\n'},
 13: {'x0': 436.1, 'x1': 468.54, 'text': b'ACTUAL\n'},
 14: {'x0': 504.4, 'x1': 520.4, 'text': b'2018\n'},
 15: {'x0': 564.4, 'x1': 580.4, 'text': b'2018\n'},
 16: {'x0': 6

In [11]:
pdf[2]   #page 2

{1: {'x0': 26.4, 'x1': 67.43, 'text': b'3/9/2018\n11:24:52AM\n'},
 2: {'x0': 26.4,
  'x1': 166.79,
  'text': b'04103 - Archie R. Cole Middle School\n'},
 3: {'x0': 310.95, 'x1': 468.0, 'text': b'EAST GREENWICH PUBLIC SCHOOL\n'},
 4: {'x0': 284.95,
  'x1': 493.88,
  'text': b'SCHOOL PROGRAM FY2019 PROPOSED BUDGET \n'},
 5: {'x0': 696.1,
  'x1': 770.52,
  'text': b'Page 2 of 13\nReport 120A\nIncl Non-Discretionary\n'},
 6: {'x0': 33.15, 'x1': 49.6, 'text': b'Fund\n'},
 7: {'x0': 65.15, 'x1': 77.59, 'text': b'Loc\n'},
 8: {'x0': 93.6, 'x1': 109.15, 'text': b'Dept\n'},
 9: {'x0': 123.85,
  'x1': 302.83,
  'text': b'Prog Gr/Sub Class Object Account Description\n'},
 10: {'x0': 381.4, 'x1': 397.4, 'text': b'2016\n'},
 11: {'x0': 373.1, 'x1': 405.54, 'text': b'ACTUAL\n'},
 12: {'x0': 444.4, 'x1': 460.4, 'text': b'2017\n'},
 13: {'x0': 436.1, 'x1': 468.54, 'text': b'ACTUAL\n'},
 14: {'x0': 504.4, 'x1': 520.4, 'text': b'2018\n'},
 15: {'x0': 564.4, 'x1': 580.4, 'text': b'2018\n'},
 16: {'x0': 6